In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets,transforms, models
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py



In [ ]:
data_dir="../input/dogs-vs-cats/train"
transform = transforms.Compose([transforms.Resize(255),
                                transforms.RandomRotation(30),
                               transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                               transforms.ToTensor(),
                               transforms.Normalize([0.5,0.5,0.5],
                                                   [0.5,0.5,0.5])])
trainset = datasets.ImageFolder(data_dir,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)

testdata_dir = '../input/dogs-vs-cats/test'
testset = datasets.ImageFolder(testdata_dir,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=64,shuffle=True)



In [ ]:
import helper
images, labels = next(iter(testloader))
fig, axes = plt.subplots(figsize=(10,4),ncols=4)
for ii in range(4):
    ax=axes[ii]
    helper.imshow(images[ii],ax=ax,normalize=False)


In [ ]:
from torch import nn 
from torch import optim
#using Densenet
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained=True)
#freeze parameters so we don't propagate through them
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(nn.Linear(1024,256),
                                nn.ReLU(),
                                nn.Dropout(0.2),
                                nn.Linear(256,2),
                                nn.LogSoftmax(dim=1) 
                                )
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr=0.01)
model.to(device)


In [ ]:
print(device)

In [ ]:
epochs = 1
steps = 0
running_loss=0
print_every=5

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps +=1
        inputs, labels = inputs.to(device), labels.to(device)
        
        logps = model.forward(inputs)
        loss = criterion(logps,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if steps%%print_every ==0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    #calculate accuracy
                    ps =torch.exp(logps)
                    top_p, top_class = ps.topk(1,dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
        
        